In [6]:
#importing packages
import pandas as pd
import numpy as np
import datetime as dt
from datetime import timedelta
import pandas_datareader as pdr
import seaborn as sns
import matplotlib.pyplot as plt
import bs4 as bs
import requests
from IPython.display import clear_output
import pickle
import os
from sklearn.model_selection import GridSearchCV
import yfinance as yf
import json
import time
sns.set()
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import BatchNormalization
import datetime as dt
from sklearn import model_selection
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout

In [7]:
def get_data(ticker, years, show):
    df = pdr.get_data_yahoo(ticker, start = dt.datetime(2021 - years,1,1), end = (dt.date.today() - timedelta(days=1)))
    plt.plot(df.reset_index().loc[:, 'Date'], df[['High', 'Low', 'Open', 'Close']], color = 'red', label = f'Real {ticker} Stock Price')
    plt.title(f'{ticker} Stock Price Prediction')
    plt.xlabel('Time')
    plt.ylabel(f'{ticker} Stock Price')
    plt.legend()
    plt.gcf().set_size_inches(20,10)
    if show:
        plt.show()
    else:
        plt.ioff()
    return df

In [8]:
def set_SMA(df, num_days):
    return df['Close'].rolling(window = num_days, min_periods=0).mean()

def Wilder(data, periods):
    start = np.where(~np.isnan(data))[0][0] #Check if nans present in beginning
    Wilder = np.array([np.nan]*len(data))
    Wilder[start+periods-1] = data[start:(start+periods)].mean() #Simple Moving Average
    for i in range(start+periods,len(data)):
        Wilder[i] = (Wilder[i-1]*(periods-1) + data[i])/periods #Wilder Smoothing
    return(Wilder)

def set_ATR(df, num_days):
    prev_close = df['Close'].shift(1)
    TR = np.maximum((df['High'] - df['Low']), 
                        np.maximum(abs(df['High'] - prev_close), 
                        abs(prev_close - df['Low'])))
    
    return Wilder(TR, num_days)

def set_STOCH(df, num_days):
    Lowest = df['Low'].rolling(window = num_days, min_periods=0).min()
    High = df['High'].rolling(window = num_days, min_periods=0).max()

    Stochastic = ((df['Close'] - Lowest)/(High - Lowest))*100

    Stochastic_D = Stochastic.rolling(window = num_days).mean()
    return Stochastic_D

def set_MACD(df, num_days):
    small_days = df['Close'].ewm(span=(num_days - 6), adjust=False, min_periods=0).mean()
    large_days = df['Close'].ewm(span=(num_days + 6), adjust=False, min_periods=0).mean()
    return large_days - small_days

def set_BBANDS(df, num_days):
    #Bollinger Bands
    Mean = df['Close'].rolling(window=(num_days + 6), min_periods=0).mean()
    SD = df['Close'].rolling(window=(num_days + 6), min_periods=0).std()
    Upperband = Mean + 2*SD
    Lowerband = Mean - 2*SD
    return (Upperband + Lowerband)/2

In [9]:
def build_network(ticker, years, days_past = 50, num_epochs = 50, show = True):
    df = get_data(ticker, years, show)
    df['SMA'] = set_SMA(df, days_past)
    df['Stoch'] = set_STOCH(df, days_past)
    df['MACD'] = set_MACD(df, days_past)
    df['BBANDS'] = set_BBANDS(df, days_past)
    
    df = df.dropna()
    df = df.drop('Volume', axis=1).drop('Adj Close', axis=1)
    
    train_size = int(df.shape[0] * 0.75)
    df_train = df[:train_size]
    df_target = df_train[['High', 'Low', 'Open', 'Close']]
    
    sc = MinMaxScaler(feature_range = (0, 1))
    target_set=df_target.values
    train_set=df_train.values
    training_set_scaled = sc.fit_transform(train_set)
    target_set_scaled = sc.fit_transform(target_set)
    
    X_train = []
    y_train = []
    for i in range(days_past,len(train_set)):
        X_train.append(training_set_scaled[i-50:i,:])
        y_train.append(target_set_scaled[i,:])

    X_train, y_train = np.array(X_train), np.array(y_train)
    if show:
        print("Shape of X_train data: " + str(X_train.shape))
        print("Shape of y_train data: " + str(y_train.shape))
    
    mod=Sequential()
    mod.add(LSTM(units = 64, return_sequences = True, input_shape = (X_train.shape[1], X_train.shape[2])))
    mod.add(Dropout(0.2))
    mod.add(LSTM(units = 64, return_sequences = True))
    mod.add(Dropout(0.1))

    mod.add((LSTM(units = 64)))
    mod.add(Dropout(0.1))
    mod.add((Dense(units = 16, activation='tanh')))
    mod.add((Dense(units = 4, activation='tanh')))
    mod.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy','mean_squared_error'])
    if show:
        mod.summary()
    
    if show:    
        mod.fit(X_train, y_train, epochs = num_epochs, batch_size = 32)
    else:
        mod.fit(X_train, y_train, epochs = num_epochs, batch_size = 32, verbose=0)
    
    df_test=df[train_size - 50:]
    test_set=df_test.values
    test_set_scaled = sc.fit_transform(test_set)
    
    df_pred = df_test.copy()
    df_pred = df_pred[50:]
    
    X_test = []

    for i in range(50, len(test_set)):
        X_test.append(test_set_scaled[i-50:i,:])

    X_test= np.array(X_test)
    
    predicted_stock_price = mod.predict(X_test)
    
    predicted_stock_price = predicted_stock_price.T
    
    for i in range(4):
        predicted_stock_price[i] = predicted_stock_price[i] * (1 / sc.scale_[i]) + sc.data_min_[i]

    predicted_stock_price = predicted_stock_price.T
    
    plt.plot(df.reset_index().loc[train_size:, 'Date'],df_pred[['High', 'Low', 'Open', 'Close']], color = 'red', label = f'Real {ticker} Stock Price')
    plt.plot(df.reset_index().loc[train_size:, 'Date'],predicted_stock_price.T[0], color = 'blue', label = f'Predicted High {ticker} Stock Price')
    plt.plot(df.reset_index().loc[train_size:, 'Date'],predicted_stock_price.T[1], color = 'blue', label = f'Predicted Low {ticker} Stock Price')
    plt.plot(df.reset_index().loc[train_size:, 'Date'],predicted_stock_price.T[2], color = 'blue', label = f'Predicted Open {ticker} Stock Price')
    plt.plot(df.reset_index().loc[train_size:, 'Date'],predicted_stock_price.T[3], color = 'blue', label = f'Predicted Close {ticker} Stock Price')
    plt.title(f'{ticker} Stock Price Prediction')
    plt.xlabel('Time')
    plt.ylabel(f'{ticker} Stock Price')
    plt.legend()
    plt.gcf().set_size_inches(20,10)
    if show:
        plt.show()
    else:
        plt.ioff()
    
    vals = ['High', 'Low', 'Open', 'Close']
    for i in range(len(vals)):
        plt.plot(df.reset_index().loc[train_size:, 'Date'],df_pred[vals[i]], color = 'red', label = f'Real {vals[i]} {ticker} Stock Price')
        plt.plot(df.reset_index().loc[train_size:, 'Date'],predicted_stock_price.T[i], color = 'blue', label = f'Predicted {vals[i]} {ticker} Stock Price')
        plt.title(f'{ticker} {vals[i]} Stock Price Prediction')
        plt.xlabel('Time')
        plt.ylabel(f'{ticker} {vals[i]} Stock Price')
        plt.legend()
        plt.gcf().set_size_inches(20,10)
        if show:
            plt.show()
        else:
            plt.ioff()
            plt.close()
        
    def predict_next_day(df, preds, day):
    
    # predict new values    
        new_preds = mod.predict(preds)

        # scale
        for i in range(4):
            new_preds[0][i] = new_preds[0][i] * (1 / sc.scale_[i]) + sc.data_min_[i]   

        # add to df
        new_row = df.iloc[-1]
        new_row.name = day
        df = df.append(new_row)
        df.iloc[-1, 4:] = 0
        for i in range(4):
            df.iloc[-1, i] = new_preds[0][i]

        # calculate technical indicators
        df['SMA'] = set_SMA(df, 50)
        df['Stoch'] = set_STOCH(df, 50)
        df['MACD'] = set_MACD(df, 50)
        df['BBANDS'] = set_BBANDS(df, 50)

        # add ti to new_preds
        new_preds = np.append(new_preds[0], df.iloc[-1, 4:])

        # update and return preds
        preds = preds[0][1:]
        new_preds = np.array(new_preds)
        new_preds = np.reshape(new_preds, (1, 8))
        preds = np.append(preds, new_preds, axis=0)
        preds = np.reshape(preds, (1, preds.shape[0], preds.shape[1]))

        return preds, df
    
    def predict_x_days(df, preds, x):
        today = dt.date.today()

        for i in range(x):
            day = today + timedelta(days = i)
            preds, df = predict_next_day(df, preds, day)

        return df
    
    def make_prediction(days):
        df_extrapolated = df.copy()
        preds = X_test[-1:]
        df_extrapolated = predict_x_days(df_extrapolated, preds, days)
        return df_extrapolated.tail(days + 1)
    
    return make_prediction
    

In [10]:
def get_percent_change(ticker):
    df = {}
    predict_future = build_network(ticker, 5, show=False)
    df_extrapolate = predict_future(30)
    def percent_change(a, b):
        return ((b-a)/a)*100
    cur_close = df_extrapolate.iloc[0,:]['Close']
    df['1 day'] = percent_change(cur_close, df_extrapolate.iloc[1,:]['Close'])
    df['2 day'] = percent_change(cur_close, df_extrapolate.iloc[2,:]['Close'])
    df['5 day'] = percent_change(cur_close, df_extrapolate.iloc[5,:]['Close'])
    df['15 day'] = percent_change(cur_close, df_extrapolate.iloc[15,:]['Close'])
    df['30 day'] = percent_change(cur_close, df_extrapolate.iloc[30,:]['Close'])
    return df